# This script generates Data representable by the netherlands it is syntethic data that mirrors the netherlands tax payers and its sectors based on the open source data we could find out there

In [1]:
import random
import csv

# =========================================================
# CONFIG – GLOBALE SETTINGS
# =========================================================

N_RECORDS = 200000  # meer data voor AI / Mesa; pas aan naar wens

# Verdeling van grootteklassen – grofweg gebaseerd op NL: heeeel veel micro
GROOTTEKLASSEN = ["micro", "klein", "midden"]
GROOTTE_PROBS = [0.82, 0.14, 0.04]  # ~80+% micro, kleine en middelgroot minder

# Sectoren – ruwe afspiegeling Nederlandse bedrijvenpopulatie
SECTORS = [
    "Detailhandel",
    "Horeca",
    "Bouw",
    "Landbouw & Visserij",
    "Industrie",
    "Vervoer & Logistiek",
    "Informatie & ICT",
    "Financiële dienstverlening",
    "Zakelijke diensten",
    "Zorg & Welzijn",
    "Onderwijs & Overheid",
    "Creatieve industrie & Media",
    "Persoonlijke dienstverlening",
]

# Sectorverdeling (ongeveer, niet exact CBS maar in de geest daarvan)
SECTOR_PROBS = [
    0.16,  # Detailhandel
    0.07,  # Horeca
    0.11,  # Bouw
    0.04,  # Landbouw & Visserij
    0.06,  # Industrie
    0.06,  # Vervoer & Logistiek
    0.07,  # Informatie & ICT
    0.04,  # Financiële dienstverlening
    0.18,  # Zakelijke diensten
    0.08,  # Zorg & Welzijn
    0.03,  # Onderwijs & Overheid (zakelijke varianten)
    0.05,  # Creatieve industrie & Media
    0.05,  # Persoonlijke dienstverlening
]

# Gedetailleerde branches per sector
BUSINESS_TYPES = {
    "Horeca": [
        "Kebabzaak",
        "Snackbar",
        "Grillroom",
        "Restaurant",
        "Café-bar",
        "Lunchroom",
        "IJssalon",
    ],
    "Detailhandel": [
        "Supermarkt",
        "Kledingwinkel",
        "Elektronicazaak",
        "Boekenwinkel",
        "Drogisterij",
        "Speciaalzaak delicatessen",
        "Webshop",
    ],
    "Bouw": [
        "Schildersbedrijf",
        "Aannemersbedrijf",
        "Tegelzetter",
        "Loodgietersbedrijf",
        "Elektra-installatiebedrijf",
    ],
    "Landbouw & Visserij": [
        "Melkveebedrijf",
        "Akkerbouwbedrijf",
        "Tuinbouwbedrijf",
        "Varkenshouderij",
        "Visserijbedrijf",
    ],
    "Industrie": [
        "Metaalbewerkingsbedrijf",
        "Voedingsmiddelenfabriek",
        "Textielfabriek",
        "Machinebouw",
    ],
    "Vervoer & Logistiek": [
        "Transportbedrijf",
        "Koeriersdienst",
        "Distributiecentrum",
        "Taxibedrijf",
    ],
    "Informatie & ICT": [
        "Softwarebedrijf",
        "IT-consultancy",
        "Hostingprovider",
        "Managed services provider",
        "Data-analysebureau",
    ],
    "Financiële dienstverlening": [
        "Assurantiekantoor",
        "Financieel adviesbureau",
        "Vermogensbeheerder",
        "FinTech-startup",
    ],
    "Zakelijke diensten": [
        "Administratiekantoor",
        "Accountantskantoor",
        "Marketingbureau",
        "HR-adviesbureau",
        "Organisatieadviesbureau",
        "Detacheringbureau",
    ],
    "Zorg & Welzijn": [
        "Huisartsenpraktijk",
        "Fysiotherapiepraktijk",
        "Thuiszorgorganisatie",
        "Psychologenpraktijk",
        "Tandartspraktijk",
    ],
    "Onderwijs & Overheid": [
        "Particulier opleidingsinstituut",
        "Taalinstituut",
        "Adviesorgaan overheid",
        "Onderwijsadviesbureau",
    ],
    "Creatieve industrie & Media": [
        "Reclamebureau",
        "Fotostudio",
        "Videoproductiebedrijf",
        "Ontwerpstudio",
        "Game-studio",
    ],
    "Persoonlijke dienstverlening": [
        "Nagelsalon",
        "Kapsalon",
        "Schoonheidssalon",
        "Barbershop",
        "Tattoo- en piercingstudio",
    ],
}

RECHTSVORMEN = ["eenmanszaak", "BV", "VOF", "maatschap"]
RECHTSVORM_PROBS = [0.50, 0.30, 0.15, 0.05]  # veel eenmanszaken in NL

# Regio’s in NL
REGIOS = ["Randstad", "Noord", "Oost", "Zuid", "Midden"]
REGIO_PROBS = [0.4, 0.12, 0.18, 0.18, 0.12]  # Randstad relatief groot

# Digi-niveau (sterk gesimuleerd)
DIGI_LEVELS = ["laag", "middel", "hoog"]
DIGI_PROBS_OVERALL = [0.35, 0.40, 0.25]

# Digitalisering per sector (extra bias)
DIGI_SECTOR_BIAS = {
    "Informatie & ICT": (0.1, 0.3, 0.6),
    "Zakelijke diensten": (0.2, 0.45, 0.35),
    "Detailhandel": (0.35, 0.40, 0.25),
    "Horeca": (0.45, 0.40, 0.15),
    "Bouw": (0.45, 0.40, 0.15),
    "Persoonlijke dienstverlening": (0.5, 0.35, 0.15),
}

# Gemiddelde winstmarges per sector (min, max) als fractie van omzet
SECTOR_MARGIN_RANGES = {
    "Detailhandel": (0.03, 0.10),
    "Horeca": (0.02, 0.08),
    "Bouw": (0.04, 0.12),
    "Landbouw & Visserij": (0.02, 0.08),
    "Industrie": (0.05, 0.15),
    "Vervoer & Logistiek": (0.03, 0.10),
    "Informatie & ICT": (0.10, 0.25),
    "Financiële dienstverlening": (0.15, 0.30),
    "Zakelijke diensten": (0.10, 0.22),
    "Zorg & Welzijn": (0.04, 0.12),
    "Onderwijs & Overheid": (0.03, 0.08),
    "Creatieve industrie & Media": (0.05, 0.18),
    "Persoonlijke dienstverlening": (0.05, 0.20),
}

# Sectoren met relatief hoger fiscaal/administratief risico
HIGHER_RISK_SECTORS = {
    "Horeca",
    "Detailhandel",
    "Bouw",
    "Vervoer & Logistiek",
    "Persoonlijke dienstverlening",
}

# Specifieke branches met extra risico (veel contant / moeilijk controleerbaar)
HIGHER_RISK_BUSINESSES = {
    "Kebabzaak",
    "Snackbar",
    "Grillroom",
    "Nagelsalon",
    "Kapsalon",
    "Barbershop",
    "Schoonheidssalon",
    "Tattoo- en piercingstudio",
    "Taxibedrijf",
}

# =========================================================
# HULPFUNCTIES
# =========================================================

def weighted_choice(options, weights):
    """Kies één optie op basis van gewichten."""
    return random.choices(options, weights=weights, k=1)[0]

def sample_jaaromzet(grootteklasse):
    """Schat globale omzet-range per grootteklasse (sterk gesimplificeerd)."""
    if grootteklasse == "micro":
        # 80k - 400k
        return random.randint(80_000, 400_000)
    elif grootteklasse == "klein":
        # 400k - 2,5M
        return random.randint(400_000, 2_500_000)
    else:  # midden
        # 2,5M - 20M
        return random.randint(2_500_000, 20_000_000)

def sample_aantal_werknemers(grootteklasse):
    """
    Simpele verdeling in lijn met EU-definities:
    - micro 1–9
    - klein 10–49
    - midden 50–249
    """
    if grootteklasse == "micro":
        return random.randint(1, 9)
    elif grootteklasse == "klein":
        return random.randint(10, 49)
    else:
        return random.randint(50, 249)

def sample_winst(omzet, sector):
    """Bepaal winst voor belasting op basis van sector-marge."""
    low, high = SECTOR_MARGIN_RANGES.get(sector, (0.05, 0.15))
    marge = random.uniform(low, high)
    winst = int(omzet * marge)
    return winst

def bereken_verschuldigde_belasting(winst, rechtsvorm):
    """
    Zeer simpele benadering:
    - eenmanszaak/VOF/maatschap: inkomstenbelasting → ca. 30% van winst
    - BV: vennootschapsbelasting: 19% tot 200k, daarboven 25%
    """
    if winst <= 0:
        return 0

    if rechtsvorm in {"eenmanszaak", "VOF", "maatschap"}:
        return int(winst * 0.30)
    else:  # BV
        if winst <= 200_000:
            return int(winst * 0.19)
        else:
            lower = 200_000 * 0.19
            upper = (winst - 200_000) * 0.25
            return int(lower + upper)

def sample_branche(sector):
    """Kies een gedetailleerd bedrijfstype binnen de sector."""
    choices = BUSINESS_TYPES.get(sector)
    if not choices:
        return sector
    return random.choice(choices)

def sample_regio():
    return weighted_choice(REGIOS, REGIO_PROBS)

def sample_digitaliseringsniveau(sector):
    """
    Sectorafhankelijk digi-niveau:
    - als sector in DIGI_SECTOR_BIAS → gebruik die verdeling
    - anders globale verdeling
    """
    if sector in DIGI_SECTOR_BIAS:
        probs = DIGI_SECTOR_BIAS[sector]
    else:
        probs = DIGI_PROBS_OVERALL
    return weighted_choice(DIGI_LEVELS, probs)

def sample_heeft_fiscaal_adviseur(grootteklasse, digitaliseringsniveau):
    """
    Kans op fiscaal adviseur:
    - hoger bij BV en grotere bedrijven (hier alleen grootteklasse + digi als proxies)
    """
    base = 0.4
    if grootteklasse == "midden":
        base += 0.3
    elif grootteklasse == "klein":
        base += 0.15

    if digitaliseringsniveau == "hoog":
        base += 0.1  # vaak ook meer uitbesteed/advies

    base = max(0.05, min(0.95, base))
    return "ja" if random.random() < base else "nee"

def sample_contant_intensief(sector, branche):
    """
    Bepaalt of de onderneming contant-intensief is.
    Contant-intensieve branches krijgen vaker 'ja'.
    """
    if branche in HIGHER_RISK_BUSINESSES:
        return "ja"
    if sector in {"Detailhandel", "Horeca", "Persoonlijke dienstverlening"}:
        return "ja" if random.random() < 0.6 else "nee"
    return "ja" if random.random() < 0.1 else "nee"

def sample_eerdere_controle():
    """
    Eenvoudige verdeling voor eerdere controle:
    de meeste bedrijven hebben geen boekenonderzoek gehad.
    """
    rnd = random.random()
    if rnd < 0.75:
        return "geen"
    elif rnd < 0.93:
        return "administratief"
    else:
        return "boekenonderzoek"

def bepaal_risico_en_aangifte(sector, grootteklasse, branche_detail,
                              digitaliseringsniveau, heeft_fiscaal_adviseur,
                              contant_intensief, eerdere_controle):
    """
    Bepaalt:
    - belasting_betaald_status
    - aangifte_juisteid
    - aangifte_tijdigheid
    - risicocategorie
    - historisch_risico_score

    Gebaseerd op:
    - sector/branche (contant risico)
    - grootteklasse
    - digitalisering
    - fiscaal adviseur
    - eerdere controle
    """

    # Basisrisico score 0–1
    base_risk = 0.20  # laag begin

    # Sector/branche
    if sector in HIGHER_RISK_SECTORS:
        base_risk += 0.20
    if branche_detail in HIGHER_RISK_BUSINESSES:
        base_risk += 0.10

    # Grootteklasse – micro vaak hogere kans op administratieve chaos
    if grootteklasse == "micro":
        base_risk += 0.10
    elif grootteklasse == "midden":
        base_risk -= 0.05

    # Contant-intensief
    if contant_intensief == "ja":
        base_risk += 0.10

    # Digitalisering – hoger digi-niveau → lager risico
    if digitaliseringsniveau == "hoog":
        base_risk -= 0.10
    elif digitaliseringsniveau == "laag":
        base_risk += 0.05

    # Fiscaal adviseur – verlaagt risico
    if heeft_fiscaal_adviseur == "ja":
        base_risk -= 0.10
    else:
        base_risk += 0.05

    # Eerdere controles – als boekenonderzoek is geweest, kans dat risico gedaald is
    if eerdere_controle == "boekenonderzoek":
        base_risk -= 0.10
    elif eerdere_controle == "administratief":
        base_risk -= 0.03

    # Clamp
    base_risk = max(0.05, min(0.90, base_risk))

    # Historisch risico-score bewaren (kan je in Mesa gebruiken als startwaarde)
    historisch_risico_score = round(base_risk, 3)

    # Kans op grote en kleine fouten
    p_grote_fout = base_risk * 0.4
    p_kleine_fout = base_risk * 0.6
    p_geen_fout = 1.0 - (p_grote_fout + p_kleine_fout)
    if p_geen_fout < 0:
        p_geen_fout = 0

    r1 = random.random()
    if r1 < p_grote_fout:
        aangifte_juisteid = "foutief_groot"
    elif r1 < p_grote_fout + p_kleine_fout:
        aangifte_juisteid = "foutief_klein"
    else:
        aangifte_juisteid = "correct"

    # Tijdigheid – gekoppeld aan base_risk
    r2 = random.random()
    if r2 < base_risk * 0.25:
        aangifte_tijdigheid = "geen_aangifte"
    elif r2 < base_risk * 0.7:
        aangifte_tijdigheid = "te_laat"
    else:
        aangifte_tijdigheid = "op_tijd"

    # Betaalstatus
    r3 = random.random()
    if aangifte_juisteid == "foutief_groot" or aangifte_tijdigheid == "geen_aangifte":
        if r3 < 0.45:
            belasting_betaald_status = "niet_betaald"
        elif r3 < 0.85:
            belasting_betaald_status = "gedeeltelijk_betaald"
        else:
            belasting_betaald_status = "volledig_betaald"
    elif aangifte_juisteid == "foutief_klein":
        if r3 < 0.12:
            belasting_betaald_status = "niet_betaald"
        elif r3 < 0.40:
            belasting_betaald_status = "gedeeltelijk_betaald"
        else:
            belasting_betaald_status = "volledig_betaald"
    else:  # correct
        if r3 < 0.04:
            belasting_betaald_status = "gedeeltelijk_betaald"
        else:
            belasting_betaald_status = "volledig_betaald"

    # Definitieve risicocategorie – op basis van historisch risico plus gedrag
    risk_score = historisch_risico_score

    if aangifte_juisteid == "foutief_groot":
        risk_score += 0.30
    elif aangifte_juisteid == "foutief_klein":
        risk_score += 0.15

    if belasting_betaald_status == "niet_betaald":
        risk_score += 0.30
    elif belasting_betaald_status == "gedeeltelijk_betaald":
        risk_score += 0.15

    if aangifte_tijdigheid == "geen_aangifte":
        risk_score += 0.25
    elif aangifte_tijdigheid == "te_laat":
        risk_score += 0.10

    # Clamp again
    risk_score = max(0.0, min(1.0, risk_score))

    if risk_score < 0.35:
        risicocategorie = "laag"
    elif risk_score < 0.7:
        risicocategorie = "middel"
    else:
        risicocategorie = "hoog"

    return (
        belasting_betaald_status,
        aangifte_juisteid,
        aangifte_tijdigheid,
        risicocategorie,
        historisch_risico_score,
    )

# =========================================================
# DATAGENERATIE
# =========================================================

def generate_mkb_dataset(n_records: int):
    records = []

    for i in range(1, n_records + 1):
        bedrijf_id = f"MKB{str(i).zfill(5)}"

        grootteklasse = weighted_choice(GROOTTEKLASSEN, GROOTTE_PROBS)
        sector = weighted_choice(SECTORS, SECTOR_PROBS)
        rechtsvorm = weighted_choice(RECHTSVORMEN, RECHTSVORM_PROBS)

        jaaromzet = sample_jaaromzet(grootteklasse)
        branche_detail = sample_branche(sector)
        regio = sample_regio()
        aantal_werknemers = sample_aantal_werknemers(grootteklasse)
        digitaliseringsniveau = sample_digitaliseringsniveau(sector)
        heeft_fiscaal_adviseur = sample_heeft_fiscaal_adviseur(
            grootteklasse, digitaliseringsniveau
        )
        contant_intensief = sample_contant_intensief(sector, branche_detail)
        eerdere_controle = sample_eerdere_controle()

        winst = sample_winst(jaaromzet, sector)
        verschuldigde = bereken_verschuldigde_belasting(winst, rechtsvorm)

        (
            belasting_betaald_status,
            aangifte_juisteid,
            aangifte_tijdigheid,
            risicocategorie,
            historisch_risico_score,
        ) = bepaal_risico_en_aangifte(
            sector,
            grootteklasse,
            branche_detail,
            digitaliseringsniveau,
            heeft_fiscaal_adviseur,
            contant_intensief,
            eerdere_controle,
        )

        record = {
            "bedrijf_id": bedrijf_id,
            "grootteklasse": grootteklasse,
            "aantal_werknemers": aantal_werknemers,
            "sector_sbi_hoofdcategorie": sector,
            "branche_detail": branche_detail,
            "regio": regio,
            "rechtsvorm": rechtsvorm,
            "jaaromzet": jaaromzet,
            "winst_voor_belasting": winst,
            "verschuldigde_belasting": verschuldigde,
            "belasting_betaald_status": belasting_betaald_status,
            "aangifte_juisteid": aangifte_juisteid,
            "aangifte_tijdigheid": aangifte_tijdigheid,
            "risicocategorie": risicocategorie,
            "digitaliseringsniveau": digitaliseringsniveau,
            "heeft_fiscaal_adviseur": heeft_fiscaal_adviseur,
            "contant_intensief": contant_intensief,
            "eerdere_controle": eerdere_controle,
            "historisch_risico_score": historisch_risico_score,
        }

        records.append(record)

    return records

# =========================================================
# RUN & SAVE
# =========================================================

if __name__ == "__main__":
    data = generate_mkb_dataset(N_RECORDS)
    output_file = "synthetic_mkb_belasting_nl_representatief.csv"

    fieldnames = list(data[0].keys())
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter=";")
        writer.writeheader()
        writer.writerows(data)

    print(f"Gegenereerd: {len(data)} records → {output_file}")


Gegenereerd: 200000 records → synthetic_mkb_belasting_nl_representatief.csv


# Begin modeling mesa ai agents